# Arduino ROS control Sensors and Actuators

Remind that we want to develop a HW structure with RaspberryPi3 and Arduino 

<img src="./Images/2_nodes_schematics_01_chatter.png">

## Arduino Publisher & Subscriber

We will have to program Publishers and Subscribers using C and Arduino IDE

Rosserial Tutorials: http://wiki.ros.org/rosserial_arduino/Tutorials

Publisher template:https://www.intorobotics.com/template-for-a-ros-publisher-using-rosserial-on-arduino/

In [ ]:
#include <ros.h>
ros::NodeHandle nh;

std_msgs::String str_msg;
ros::Publisher pub("any_topic", &str_msg);

void setup(){
...
nh.initNode();
nh.advertise(pub);
...
}

void loop()
{
pub.publish( &str_msg );
nh.spinOnce();
}

Exemple "Hello World"

In [ ]:
/*
 * rosserial Publisher Example
 * Prints "hello world!"
 */

// Use the following line if you have a Leonardo or MKR1000
//#define USE_USBCON

#include <ros.h>
#include <std_msgs/String.h>

ros::NodeHandle nh;

std_msgs::String str_msg;
ros::Publisher chatter("chatter", &str_msg);

char hello[13] = "hello world!";

void setup()
{
  nh.initNode();
  nh.advertise(chatter);
}

void loop()
{
  str_msg.data = hello;
  chatter.publish( &str_msg );
  nh.spinOnce();
  delay(1000);
}


In [ ]:
rosrun rosserial_python serial_node.py /dev/ttyACM0

Subscriber template:https://www.intorobotics.com/template-for-a-ros-subscriber-using-rosserial-on-arduino/

In [ ]:
#include <ros.h>
#include <std_msgs/THE_TYPE_OF_THE_MESSAGE_YOU_SUBSCRIBER>

//create the ros node nh. The node will be used to publish to Arduino
ros::NodeHandle nh;


void messageCb(const std_msgs::MESSAGE_TYPE& msg)
{
  do_domething with msg.data;
 
}

ros::Subscriber<std_msgs::MESSAGE_TYPE> sub("THE_TOPIC_THAT_SUBSCRIBER", &messageCb);

void setup()
{
  nh.initNode();
  nh.subscribe(sub);
}

void loop()
{
  nh.spinOnce();
  delay(10);
}

Exemple "Blink"

In [ ]:
/*
 * rosserial Subscriber Example
 * Blinks an LED on callback
 */

#include <ros.h>
#include <std_msgs/Empty.h>

ros::NodeHandle nh;

void messageCb( const std_msgs::Empty& toggle_msg){
  digitalWrite(13, HIGH-digitalRead(13));   // blink the led
}

ros::Subscriber<std_msgs::Empty> sub("toggle_led", &messageCb );

void setup()
{
  pinMode(13, OUTPUT);
  nh.initNode();
  nh.subscribe(sub);
}

void loop()
{
  nh.spinOnce();
  delay(1);
}


In [ ]:
rosrun rosserial_python serial_node.py /dev/ttyACM0

In [ ]:
rostopic pub toggle_led std_msgs/Empty --once

to publish and latch message for 3s

In [ ]:
rostopic pub toggle_led std_msgs/Empty "{}"

to publish and latch message. ctrl C to terminate

## Exercise: Closed loop system

We are going to develop a closed loop system between Arduino and our PC

<img src="./Images/2_nodes_schematics_01_chatter.png">

In Arduino will develop a ROS node for Subscriber & Publisher according to the exemple in ros-lib: pubsub.ino

In [ ]:
/*
 * rosserial PubSub Example
 * Prints "Toggle LED!" and toggles led
 */

#include <ros.h>
#include <std_msgs/String.h>
#include <std_msgs/Empty.h>

ros::NodeHandle  nh;


void messageCb( const std_msgs::Empty& toggle_msg){
  digitalWrite(13, HIGH-digitalRead(13));   // blink the led
}

ros::Subscriber<std_msgs::Empty> sub("toggle_led", messageCb );

std_msgs::String str_msg;
ros::Publisher chatter("chatter", &str_msg);

char toggle[11] = "Toggle LED!";

void setup()
{
  pinMode(13, OUTPUT);
  nh.initNode();
  nh.advertise(chatter);
  nh.subscribe(sub);
}

void loop()
{
  str_msg.data = toggle;
  chatter.publish( &str_msg );
  nh.spinOnce();
  delay(1000);
}

Explanation

The code imports the ros_lib library and standard ROS messages for std_msgs::String and std_msgs::Empty. Variables are declared for storing the ROS data types and Arduino board pins.

The code declares a publisher named "chatter" which publishes str_msg of std_msgs::String data type to the /chatter topic. Similarly, a subscriber named "sub" subscribes to the "/toggle_led" topic which gets an std_msgs::Empty data type.

A ROS node handle is declared and then initialized in the setup() method. The node handle advertises the publishers and subscribes to the topic.

The messageCb() is the method called when data is received on the Arduino subscriber node. The callback method checks if the ROS topic data is HIGH or not and accordingly toggles the connected LEDs.

Finally, the loop() method runs an infinite loop to publish the toggle string data it to ROS. The node handle makes a call to spinOnce() to handle communication. This loop is executed every 1 second.

In a first test, we will open a new terminal in a PC to:
- echo the Published message in the topic /chatter
- pub and empty message to toggle the led

In [ ]:
rosrun rosserial_python serial_node.py /dev/ttyACM0

In [ ]:
rostopic list

In [ ]:
rostopic echo /chatter

In [ ]:
rostopic pub toggle_led std_msgs/Empty --once

<img src="./Images/02_pubsub_PC.png">

In our PC we will develop a ROS node for Subscriber & Publisher

In [ ]:
#!/usr/bin/env python
# the previous line has allways to be the first line
#remove or add the library/libraries for ROS
import rospy

#remove or add the message type
from std_msgs.msg import String, Empty

# Initialise variables
chat_msg=""
pub = None

#define function/functions to provide the required functionality
def chatterCb(msg): # msg could be any name
    global chat_msg
    chat_msg=msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", chat_msg)  
    if chat_msg=="Toggle LED!":
        toggle_msg=Empty() # message to be published
        rospy.loginfo("OK. I send Empty message to toggle LED")
        pub.publish(toggle_msg)
    else:
        rospy.loginfo("NOT a correct order")


if __name__=='__main__':

    try:
        #Add here the name of the ROS node. Node names are unique named. Here without "anonimous"!
        rospy.init_node('rUBot_ctrl_node')
        #subscribe to a topic using rospy.Subscriber class
        sub=rospy.Subscriber('/chatter', String, chatterCb)
        #publish messages to a topic using rospy.Publisher class
        pub=rospy.Publisher('/toggle_led', Empty, queue_size=1)
  
        rospy.spin()
      
    except rospy.ROSInterruptException:
        rospy.loginfo("node terminated.")

You can perform the code using the Object Oriented Programing structure:

In [ ]:
#!/usr/bin/env python

import rospy
from std_msgs.msg import Int64
from std_srvs.srv import SetBool


class NumberCounter:

    def __init__(self):
		self.counter = 0

		self.number_subscriber = rospy.Subscriber("/number", Int64, self.callback_number)

		self.pub = rospy.Publisher("/number_count", Int64, queue_size=10)

		self.reset_service = rospy.Service("/reset_counter", SetBool, self.callback_reset_counter)

	def callback_number(self, msg):
		self.counter += msg.data
		new_msg = Int64()
		new_msg.data = self.counter
		self.pub.publish(new_msg)

	def callback_reset_counter(self, req):
		if req.data:
			self.counter = 0
			return True, "Counter has been successfully reset"
		return False, "Counter has not been reset"


if __name__ == '__main__':
	rospy.init_node('number_counter')
	NumberCounter()
	rospy.spin()


In [ ]:
rosrun hw_motion_plan PubSub_LedChatter.py 

In [ ]:
rqt_graph

<img src="./Images/2_LedChatter_terminal.png">

<img src="./Images/2_rqt_graph_Led.png">

https://atadiat.com/en/e-ros-imu-and-arduino-how-to-send-to-ros/

### Launch file

we can use a launch file to start all the nodes. 
We create a launch file "LedChatter.launch" in the "hw_motion_plan" package:

In [ ]:
<launch>
    <node name="serial_node"        pkg="rosserial_python"      type="serial_node.py">
    <param name="port"              type="string"               value="/dev/ttyACM0"/>
    <param name="baud"              type="int"                  value="57600"/>
    </node>

    <node name="rUBot_ctrl_node"        pkg="hw_motion_plan"      type="PubSub_LedChatter.py">
    </node>

</launch>

# Arduino Node with sensors and actuators

Remind that we want to develop a HW structure with RaspberryPi3 and Arduino 

<img src="./Images/1_nodes_schematics.png">

Bibliography:

course ROS by Anis Koubaa: https://github.com/aniskoubaa/ros_essentials_cpp

Course ROS by M. Lukman:https://www.udemy.com/course/mastering-mobile-robot-with-ros-ardunio-car-sensors-to-ros/learn/lecture/14385996#content

### Range sensor

From the different Ultrasonic range sensors: https://www.intorobotics.com/interfacing-programming-ultrasonic-sensors-tutorials-resources/

Develop our first program for Ultrasonic range sensor HC-SC04 (or SR05):

<img src="./Images/HC_SRF05.png">

Install the library Ultrasound by Erick Simoes from the library manager.

There are some exemples wit one and several ultrasonic sensors.

we use sensor_msgs/Range data type: http://docs.ros.org/melodic/api/sensor_msgs/html/msg/Range.html

We convert the code in the frame of ROS-Publisher code: (ROS_1HC_SR04_Pub.ino)

In [ ]:
/*
 * ROS Ultrasonic 1 unit HC-SR04
*/

#include <Ultrasonic.h>
#include <ros.h>
#include <sensor_msgs/Range.h>

Ultrasonic ultrasonic(12, 13);

ros::NodeHandle nh;

sensor_msgs::Range range_msg;

ros::Publisher pub_range_ultrasound("/ultrasound", &range_msg);

void setup() {

   nh.initNode();
   nh.advertise(pub_range_ultrasound);
}

void loop() {

    range_msg.range = ultrasonic.read();
    pub_range_ultrasound.publish(&range_msg);

    nh.spinOnce();
}

### IMU 9250

### DC motors

You can use the driver L298N

you can follow the tutorial: https://www.youtube.com/watch?v=c0L4gNKwjRw

<img src="./Images/L298N.png">

<img src="./Images/L298N_connections.png">

We will use geometry_msgs/Twist: http://docs.ros.org/melodic/api/geometry_msgs/html/msg/Twist.html

Interesting codes in:

-with arduino mega: https://create.arduino.cc/projecthub/robinb/nox-a-house-wandering-robot-ros-652315
- with ESP8266: https://github.com/Reinbert/ros_diffdrive_robot
- ROS speciffic package: http://wiki.ros.org/simple_drive

We will develop our own controller using a L298N driver:

https://maker.pro/arduino/tutorial/how-to-use-arduino-with-robot-operating-system-ros

<img src="./Images/DC_motor_arduino.png">

The code in ROS is in ROS_DC_motor_rUBot.ino

The code integrated with the sensors is in:
- ROS_sensors2motor1.ino Including cmd_vel with Empty type message
- ROS_sensors2motor.ino Including cmd_vel with Twist type message
- ROS_sensors2motor2.ino Including cmd_vel with Twist type message and IKINE and functions

Note!: if we are using Arduino UNO board, the scketch uses 75% of the total flash memory available (32KB)
If we use a STM32 Nucleo 103RB, the scketch uses the 11% of the total flash memory available (128KB)

We will have to use STM32 Nucleo boards for the project!

In PC rUBot_navigation.py

In [ ]:
#!/usr/bin/env python

import rospy

from std_msgs.msg import String, Int32, Empty

# Initialise variables
range_msg=0
imu_msg=""
odom_msg=0

pub = None

#define function/functions to provide the required functionality
def rangeCb(r_msg): # msg could be any name
    global range_msg
    range_msg=r_msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", range_msg)  
    if range_msg<7:
        vel_msg=Empty() # message to be published
        rospy.loginfo("Too close!. I send Empty message to toggle LED")
        pub.publish(vel_msg)
    else:
        rospy.loginfo("Far enough!")

def imuCb(i_msg):
    global imu_msg
    imu_msg=i_msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", imu_msg)

def odomCb(o_msg):
    global odom_msg
    odom_msg=o_msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", odom_msg)
    
if __name__=='__main__':

    try:
        #Add here the name of the ROS node. Node names are unique named. Here without "anonimous"!
        rospy.init_node('rUBot_ctrl_node')
        #subscribe to a topic using rospy.Subscriber class
        sub_range=rospy.Subscriber('/range', Int32, rangeCb)
        sub_imu=rospy.Subscriber('/imu', String, imuCb)
        sub_odom=rospy.Subscriber('/odom', Int32, odomCb)
        #publish messages to a topic using rospy.Publisher class
        pub=rospy.Publisher('/cmd_vel', Empty, queue_size=10)
  
        rospy.spin()
      
    except rospy.ROSInterruptException:
        rospy.loginfo("node terminated.")

In [ ]:
rosrun hw_motion_plan rUBot_navigation.py

<img src="./Images/3_sensor2motor_terminal.png">

<img src="./Images/3_sensor2motor_rqt.png">

we can use a launch file to start all the nodes. Even we can start the "teleop_twist_keyboard" node to control the robot with the keyboard. 
We create a launch "rUBot_control.launch" file in the "hw_motion_plan" package:

In [ ]:
<launch>
    <node name="serial_node"        pkg="rosserial_python"      type="serial_node.py">
    <param name="port"              type="string"               value="/dev/ttyACM0"/>
    <param name="baud"              type="int"                  value="57600"/>
    </node>

    <node name="rUBot_ctrl_node"        pkg="hw_motion_plan"      type="rUBot_navigation.py">
    </node>

    <node name="teleop_twist_keyboard"        pkg="teleop_twist_keyboard"      type="teleop_twist_keyboard.py">
    </node>

</launch>

To start you can launch this file and the roscore will automatically start:

In [ ]:
roslaunch hw_motion_plan rUBot_control.launch